### Retreival and preprocessing of data
* Retrieves osu map and score data 
* Exports data into "data/training.npz". Exported data can be loaded with numpy.load()

In [114]:
%pip install requests
%pip install pyquery
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [115]:
import requests
import numpy as np
from tqdm import tqdm


In [116]:
api_url_1 = "https://osu.ppy.sh/api/v1"
api_url_2 = "https://osu.ppy.sh/api/v2"
client_id = "20895"
client_secret = "mB4gPoINvVP3lypJ7h85fejwrCKQHcsgIoIjYUYz"
api_key = "d396c636e41fdef810245ce181d66cfd603aa3ea"
mod_val = 1096 # Mod value based on Osu v1 api

In [1]:
# compile maps from start date to end date (in increments of 500). Then, add beatmap info and pp info to python lists

start_date = '2021-01-01'
end_date = '2023-01-01'

data_list = []
pp_list = []

while (start_date<end_date):
    params = {"k": api_key,
            "m": 0,
            "since": start_date}

    beatmaps = requests.get(f"https://osu.ppy.sh/api/get_beatmaps", params=params).json()


    next_date = beatmaps[-1]['approved_date']
    print(f"processing maps from {start_date} to {next_date}")

    for map in tqdm(beatmaps):
        if map["approved"] != '1': # non ranked maps (no pp available)
            continue
        id = map['beatmap_id']
        params = {"k": api_key,
            "b": id,
            "limit": 100,
            "mods": mod_val}
        scores = requests.get(f"https://osu.ppy.sh/api/get_scores", params=params).json()
        if scores is None or len(scores) == 0:
            continue
        first = scores[0]
        last = scores[-1]
        
        for score in [first, last]:
            perf = int(score['count300'])+int(score['countgeki'])
            mid = int(score['count100'])+int(score['countkatu'])
            low = int(score['count50'])
            miss = int(score['countmiss'])
            total = perf+mid+low+miss
            
            diff = (int(map['max_combo'])-int(score['maxcombo']))/float(score['maxcombo'])
            acc = float(300*perf+100*mid+50*low)/(300*total)
        
            single_list = [float(map['diff_overall']),
                            float(map['diff_approach']),
                            float(map['diff_aim']),
                            float(map['diff_speed']),
                            float(map['difficultyrating']),
                            float(map['bpm']),
                            float(map['max_combo']),
                            acc,
                            diff]
            data_list.append(single_list)
            pp_list.append(float(score['pp']))
    start_date = next_date


NameError: name 'api_key' is not defined

In [118]:
data_arr = np.asarray(data_list)
pp_arr = np.asarray(pp_list)
data_arr.shape

(2546, 9)

In [119]:
# export data to "data/training"
print("exporting data...")

np.savez_compressed(file=f"data/training{mod_val}", X=data_arr, y=pp_arr)

print("done!")

exporting data...
done!
